In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log-update.v1/fact/granularity=weekly/year=2012/_update_date=2020-09-19/

 




In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-qa/unified/qa.app-est-qa.v1/dimension/granularity=daily/test_trigger_date=2020-08-18/").where("test_count is null ").show()


In [0]:
%%sh


# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-raw/_obselete/store.app_est_20170101_20190101_dna_2020_0630_0707/fact/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/

# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-dna-log-pubrange.v1/fact/granularity=daily/month=2019-11/pub_range=999/device_code=ios-phone/  --recursive --human --summarize 
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-dna-log-pubrange.v3/fact/granularity=daily/month=2019-05/pub_range=10/device_code=ios-phone/

aws s3 ls s3://b2c-prod-data-pipeline-raw-store-paid/raw/store.app-est.v3/insert/_default_partition=2020-08/


In [0]:
%python
from datetime import timedelta, date, datetime
import datetime as dt
def datetime_to_string(date_time, convert_format="%Y-%m-%d"):
    return datetime.strftime(date_time, convert_format)


def string_to_datetime(date_str, convert_format=None):
    return datetime.strptime(date_str, convert_format if convert_format else "%Y-%m-%d")
print string_to_datetime("2020-01-01")

date = string_to_datetime("2020-01-01")
weekly_day_nums=5
print [ datetime_to_string(date - timedelta(days=x)) for x in range(weekly_day_nums)]


In [0]:



df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-dna-log-pubrange.v2/fact/").where("granularity = 'daily' and month = '2019-06' ")
df.createOrReplaceTempView("temp")
df_temp_publish_change = spark.sql("select sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue),  sum(est_organic_download) from temp").cache()
df_temp_publish_change.createOrReplaceTempView("publisher_change")
# spark.sql("select * from publisher_change").show()


df_date_category = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-load.v3/fact/").where("granularity = 'daily' and date between '2019-06-01' and '2019-06-30'")
df_date_category.createOrReplaceTempView("temp_date_category")
df_category_comparasion = spark.sql("select sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue), (sum(est_free_app_download) + sum(est_paid_app_download) ) as est_organic_download  from (select max(est_free_app_download) as est_free_app_download , max(est_paid_app_download) as est_paid_app_download, max(est_revenue) as est_revenue, device_code, country_code, date, app_id from temp_date_category group by device_code, country_code, date, app_id ) as prod").cache()
df_category_comparasion.createOrReplaceTempView("category_comparsion")


df_1 = spark.sql("select * from publisher_change except select * from category_comparsion")
df_2 = spark.sql("select * from category_comparsion except select * from publisher_change")
 
 
print df_1.first()
print df_2.first()

# if df_1.rdd.isEmpty():
#     print 'pass'
# if df_2.rdd.isEmpty():
#     print 'pass'


In [0]:

df_temp_publish_change.unpersist()
df_category_comparasion.unpersist()

In [0]:

df.rdd.getNumPartitions()


In [0]:


# spark.sql("select  sum(est_organic_download) from (select max(est_organic_download) as est_organic_download, device_code, country_code, date, app_id from temp_date_category group by device_code, country_code, date, app_id ) as prod ").show()


In [0]:

df_date = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/").where("granularity = 'daily' and date between '2019-08-01' and '2019-08-31'")
df_date.createOrReplaceTempView("temp_date")
spark.sql("select sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue), sum(est_paid_download), sum(est_organic_download) from temp_date ").show()


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app_est_20170101_20190101_dna_2020_0630_0707/fact/


In [0]:


# changed app
df_dna_old=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app_est_20170101_20190101_dna_2020_0630_0707/fact/").cache()
df_dna_old.createOrReplaceTempView("dna_change_old")
spark.sql("select distinct app_id, publisher_id, company_id, parent_company_id from dna_change_old").createOrReplaceTempView("dna_change_old_1")

# new dna change log
df_dna_change_log = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/").filter("update_date = '2020-07-07'").drop("_identifier", "update_date")
df_dna_change_log.createOrReplaceTempView("df_dna_change_log")
# spark.sql("select app_id, publisher_id , company_id, parent_company_id, max(start_date) from df_dna_change_log  where app_id=1337585357 group by app_id, publisher_id , company_id, parent_company_id").show()

# changed data with new app, publisher, company, parent company relationship
df_dna_new = spark.sql("select a.app_id, b.publisher_id, b.company_id, b.parent_company_id, b.start_date, b.end_date from dna_change_old_1 as a join df_dna_change_log as b on a.app_id=b.app_id  ").cache()
df_dna_new.createOrReplaceTempView("df_dna_new")

# update app est 
df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-dna-log-pubrange.v1/fact").where("granularity = 'daily' and month = '2019-06'  ").cache()
df.createOrReplaceTempView("temp")

# spark.sql("select count(*) from temp ").show()
# spark.sql("select  count(*) from df_dna_new ").show()


spark.sql('''
SELECT DISTINCT a.app_id as app_id_a, 
                a.publisher_id AS publisher_a, 
                a.company_id as ccompany_a, 
                a.parent_company_id as parent_company_a, 
                b.app_id, 
                b.publisher_id AS publisher_b, 
                b.company_id, 
                b.parent_company_id,
                b.start_date,
                b.end_date
FROM   temp a 
       JOIN df_dna_new b 
         ON a.app_id = b.app_id 
        AND a.publisher_id = b.publisher_id 
        Where (a.date >= b.start_date 
        And a.date <= b.end_date)
        Or (a.date >= b.start_date 
        And b.end_date is null)
order by  app_id_a desc
''').createOrReplaceTempView("join_result")

spark.sql("select count(1) from join_result").show()
spark.sql("select * from join_result where ccompany_a != company_id or parent_company_a!=parent_company_id ").show(200)

# spark.sql("select * from join_result").show(20)
# spark.sql("select * from df_dna_new except select distinct app_id, publisher_id, company_id, parent_company_id from temp ").show()
# spark.sql("select distinct app_id, publisher_id, parent_company_id, company_id from temp except select * from df_dna_new").show()
# |20600004844447|20200000058031|1000200000087666| 1000200000095158|20600004844447|20200000058031|1000200000087666| 1000200000095158|2014-05-08|      null|
# |20600004844447|20200000058031|1000200000087666| 1000200000095158|20600004844447|20200000058031|1000200000087666|                0|1970-01-01|2014-05-07|



In [0]:


a.company_id != b.company_id
        OR a.parent_company_id != b.parent_company_id
        And
        
        
        
df_dna_change_log_06_30 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/").filter("update_date = '2020-06-30'").drop("_identifier", "update_date").createOrReplaceTempView("df_dna_change_log_06_30")
df_dna_change_log_07_07 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/").filter("update_date = '2020-07-07'").drop("_identifier", "update_date").createOrReplaceTempView("df_dna_change_log_07_07")


spark.sql("select * from df_dna_change_log_06_30 where app_id = 1287987924").show()
spark.sql("select * from df_dna_change_log_07_07 where app_id = 1287987924").show()


In [0]:

spark.sql("select * from temp where app_id = 1287987924").show()

In [0]:

from pyspark.sql.functions import *
df_2 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.publisher-company-mapping.v1/dimension/").filter("update_date = '2020-07-07'").drop("_identifier", "update_date")
df_1 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.publisher-company-mapping.v1/dimension/").filter("update_date = '2020-06-30'").drop("_identifier", "update_date")
df_3 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/").filter("update_date = '2020-07-07'").drop("_identifier", "update_date")
df_2.createOrReplaceTempView("df2")
df_1.createOrReplaceTempView("df1")
df_3.createOrReplaceTempView("df3")
# added_df = df_2.select("publisher_id").subtract(df_1.select("publisher_id"))
diff_df_ = spark.sql("""
select * from (
select df2.*, df1.company_id as df1_company_id, df1.publisher_id as df1_publisher_id
from
df2 full outer join df1
on df2.publisher_id = df1.publisher_id
) tmp
where company_id != df1_company_id or df1_company_id is null
""")
#print diff_df_.count()
#print diff_df_.show(400, False)
diff_df_.createOrReplaceTempView("diff_df_")
diff_df = spark.sql("""
select 
df3.*
from df3 inner join diff_df_ on df3.publisher_id = diff_df_.publisher_id and df3.company_id = diff_df_.company_id
""").na.fill({"end_date": "2020-07-07"}).withColumn("start_date", when(col("start_date") == "1970-01-01", "2010-07-01").otherwise(col("start_date")))
print diff_df.count()
# diff_df.show(20, False)
# diff_df_100apps = diff_df.select("app_id").distinct().orderBy("app_id").limit(100).createOrReplaceTempView("diff_df_100apps")
diff_df.createOrReplaceTempView("diff_df")
spark.sql("select * from df2 where publisher_id=1379847699").show()
spark.sql("select * from df1 where publisher_id=1379847699").show()

spark.sql("select * from diff_df where app_id=1379847700").show()
# diff_df_100apps_data = spark.sql("""select a.* from diff_df a join diff_df_100apps b on a.app_id = b.app_id""")
# diff_df_100apps_data.createOrReplaceTempView("diff_df")
# print diff_df_100apps_data.count()
# print diff_df_100apps_data.select("app_id").distinct().count()

In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-publisher-dna-log-pubrange.v1/fact/granularity=daily/month=2019-01/pub_range=10/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher-dna-log.v1/fact/granularity=daily/


In [0]:


df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-publisher-dna-log-pubrange.v1/fact/").where("granularity = 'daily' and month = '2019-06' ")
df.createOrReplaceTempView("temp")
df_temp_publish_change = spark.sql("select sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue) from temp").cache()
df_temp_publish_change.createOrReplaceTempView("publisher_change")
# spark.sql("select * from publisher_change").show()


df_date_category = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher-dna-log.v1/fact/").where("granularity = 'daily' and date between '2019-06-01' and '2019-06-30'")
df_date_category.createOrReplaceTempView("temp_date_category")
df_category_comparasion = spark.sql("select sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue)  from temp_date_category").cache()
df_category_comparasion.createOrReplaceTempView("category_comparsion")


df_1 = spark.sql("select * from publisher_change except select * from category_comparsion")
df_2 = spark.sql("select * from category_comparsion except select * from publisher_change")
 
 
print df_1.first()
print df_2.first()


In [0]:

df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-publisher-dna-log-pubrange.v1/fact/").where("granularity = 'daily' and month = '2019-06' ")
df.createOrReplaceTempView("temp")
data = spark.sql("select * from temp limit 5").collect()
print data

In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher-dna-log.v1/fact/granularity=daily/date=2011-06-13/").show()